In [7]:
import os#
#os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH\\')
os.chdir("/rds/general/user/ma11115/home/STK_Search/STK_search")
from pathlib import Path#
import pandas as pd
import numpy as np
from stk_search.Objective_function import IP_ES1_fosc
from stk_search.utils import database_utils
import importlib

In [2]:
calculator = IP_ES1_fosc(oligomer_size =6)

In [16]:
# load the data and the precursors dataset
importlib.reload(database_utils)
df_total_path = 'data/output/Full_dataset/filtered_data150524.csv'
df_precursors_path ="data/output/Prescursor_data/precursor_data_new_clustering.pkl"
df_total_new , df_precursors= database_utils.load_data_from_file(df_total_path, df_precursors_path,add_feature_frag=True)
print(df_total_new.shape[0])

29589


In [32]:
df_precursors
def generate_list_monomer(df_precursors,oligomer_size = 6,):
    df_multi = pd.DataFrame()
    for i in range(oligomer_size):
        df_multi[f'InChIKey_{i}'] = df_precursors['InChIKey']

    return df_multi
df_elements = generate_list_monomer(df_precursors,oligomer_size = 6)

In [49]:
#split_df_elements to chunks and save them
def split_df_elements(df_elements,chunk_size = 10):
    n_chunks = int(np.ceil(df_elements.shape[0]/chunk_size))
    os.makedirs('data/dataframe_to_run_calculations',exist_ok = True)
    for i in range(n_chunks):
        df_elements.iloc[i*chunk_size:(i+1)*chunk_size].to_csv(f'data/dataframe_to_run_calculations/df_elements_{i+1}.csv',index = False)
    return n_chunks
num_iterations = split_df_elements(df_elements,chunk_size = 20)
print(num_iterations)

14


In [52]:
df_elements_1 = pd.read_csv('data/dataframe_to_run_calculations/df_elements_1.csv')
%run src/dev_scripts/Calculate_polymer_properties.py --df_path "data/dataframe_to_run_calculations/df_elements_1.csv" --oligomer_size 6

Fitness function: -1.9936456424839286 Inchikey: RVIGGSPUKFBKCN-UHFFFAOYSA-N
Error in InChIKey_0    UVCFYBOLMFIWGF-FPYGCLRLSA-N
InChIKey_1    UVCFYBOLMFIWGF-FPYGCLRLSA-N
InChIKey_2    UVCFYBOLMFIWGF-FPYGCLRLSA-N
InChIKey_3    UVCFYBOLMFIWGF-FPYGCLRLSA-N
InChIKey_4    UVCFYBOLMFIWGF-FPYGCLRLSA-N
InChIKey_5    UVCFYBOLMFIWGF-FPYGCLRLSA-N
Name: 1, dtype: object
Bad Conformer Id
Error in InChIKey_0    PQZAHLJJGPAGBD-OQFOIZHKSA-N
InChIKey_1    PQZAHLJJGPAGBD-OQFOIZHKSA-N
InChIKey_2    PQZAHLJJGPAGBD-OQFOIZHKSA-N
InChIKey_3    PQZAHLJJGPAGBD-OQFOIZHKSA-N
InChIKey_4    PQZAHLJJGPAGBD-OQFOIZHKSA-N
InChIKey_5    PQZAHLJJGPAGBD-OQFOIZHKSA-N
Name: 2, dtype: object
Bad Conformer Id
Fitness function: -1.7252836757043173 Inchikey: NXUJZUWZLUEEKQ-UHFFFAOYSA-N


In [51]:
string_to_run = "python src/dev_scripts/Calculate_polymer_properties.py --df_path 'data/dataframe_to_run_calculations/df_elements_{$PBS_ARRAY_INDEX}.csv' --oligomer_size 6"
script_qsub = "#!/bin/bash \n"+\
    "#PBS -l walltime=07:59:01 \n"+\
    f"#PBS -l select=1:ncpus={30}:mem={50}gb:avx=true \n"+\
    f"#PBS -J 1-{num_iterations} \n"+\
    " \n"+\
    "cd /rds/general/user/ma11115/home/STK_Search/STK_search \n"+\
    "module load anaconda3/personal \n"+ \
    "source activate Geom3D     \n"+\
    string_to_run
import datetime
def submit_job(script_qsub,):
    now = datetime.datetime.now()
    # print (now.strftime("%Y-%m-%d %H:%M:%S"))
    sh_file_name = (
        f"HPC_bash_script/runcalculations_{now.strftime('%M_%S_%f')}.sh"
    )
    with open(sh_file_name, "w") as text_file:
        text_file.write(script_qsub)

    os.system(f"qsub -e ./cache -o ./cache {sh_file_name}")
    return
submit_job(script_qsub,)

9431399[].pbs
